In [1]:
from geopolrisk.assessment.main import gprs_calc
from geopolrisk.assessment.utils import regions
from geopolrisk.assessment.database import Database

In [2]:
db = Database()
db.initialize()

Attempting to connect to database: C:\Users\hahnme_a\PycharmProjects\geopolrisk-py\geopolrisk\lib\world_mining_data.db



eading table/s ['Aluminium', 'Antimony', 'Arsenic', 'Asbestos', 'Baryte', 'Bau

Attempting to connect to database: C:\Users\hahnme_a\PycharmProjects\geopolrisk-py\geopolrisk\lib\wgi.db



eading table/s ['Normalized'] from the library database C:\Users\hahnme_a\Pych

Attempting to connect to database: C:\Users\hahnme_a\PycharmProjects\geopolrisk-py\geopolrisk\lib\baci.db



eading table/s ['baci_trade'] from the library database C:\Users\hahnme_a\Pych

In [3]:
# MapISOdf = db.production["Country_ISO"]
# MapISOdf["Country"].unique().tolist()

In [4]:
# db.regional = True
# print(db.regional)

In [5]:
ListofMetals = [
    260600,  # Aluminium
    261710,  # Antimony
    280480,  # Arsenic
    252400,  # Asbestos
    251110,  # Baryte
    280450,  # Boron
    810710,  # Cadmium
    261000,  # Chromium
    270111,  # Coal
    282200,  # Cobalt
    260300,  # Copper
    270900,  # Crude oil
    252910,  # Feldspar
    252922,  # Fluorspar
    811291,  # Gallium
    282560,  # Germanium
    710811,  # Gold
    250410,  # Graphite
    252010,  # Gypsum
    811291,  # Indium
    260111,  # Iron
    260700,  # Lead
    270112,  # Lignite
    283691,  # Lithium
    251910,  # Magnesium
    260200,  # Manganese
    280540,  # Mercury
    261310,  # Molybdenum
    271111,  # Natural gas
    260400,  # Nickel
    261590,  # Niobium
    711021,  # Palladium
    711011,  # Platinum
    284690,  # Rare earth
    811291,  # Rhenium
    711031,  # Rhodium
    280490,  # Selenium
    261610,  # Silver
    260900,  # Tin
    261400,  # Titanium
    261100,  # Tungsten
    261210,  # Uranium
    261590,  # Vanadium
    260800,  # Zinc
    261510,  # Zirconium
]

ListofYear = [2018, 2019, 2020, 2021, 2022]

ListofCountries = [
    757, # Switzerland
    # "EU",
]

In [6]:
# Extract all country codes from the 'Country_ISO' table
try:
    country_table = db.production["Country_ISO"] 
    ListofCountries = country_table["ISO"].tolist()
except KeyError:
    raise KeyError("Country_ISO table not found in db.production.")
except Exception as e:
    raise RuntimeError(f"Error extracting country codes: {e}")

In [7]:
try:
    gprs_calc(period=ListofYear, countries=ListofCountries, resources=ListofMetals, db=db)
    print("GeoPolRisk calculation completed successfully.")
except Exception as e:
    print(f"Error running GeoPolRisk calculation: {e}")


alculating the GeoPolRisk: 100%|█| 57600/57600 [1:15:52<00:00, 12.65iterations

Results successfully saved to C:\Users\hahnme_a\PycharmProjects\geopolrisk-py\output\results.xlsx
GeoPolRisk calculation completed successfully.
